In [1]:

'''
FWAF - Machine Learning driven Web Application Firewall
Author: Faizan Ahmad
Performance improvements: Timo Mechsner
Website: http://fsecurify.com
'''

from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import urllib.parse

import matplotlib.pyplot as plt

def loadFile(name):
    directory = str(os.getcwd())
    filepath = os.path.join(directory, name)
    with open(filepath,'r') as f:
        data = f.readlines()
    data = list(set(data))
    result = []
    for d in data:
        d = str(urllib.parse.unquote(d))   #converting url encoded data to simple string
        result.append(d)
    return result

badQueries = loadFile('badqueries.txt')
validQueries = loadFile('goodqueries.txt')


In [2]:


badQueries = list(set(badQueries))
validQueries = list(set(validQueries))
allQueries = badQueries + validQueries
yBad = [1 for i in range(0, len(badQueries))]  #labels, 1 for malicious and 0 for clean
yGood = [0 for i in range(0, len(validQueries))]
y = yBad + yGood
queries = allQueries



In [3]:

"""
TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 
여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 문서의 핵심어를 추출하거나, 
검색 엔진에서 검색 결과의 순위를 결정하거나, 문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.
"""
#TF = (용어 t가 문서에 나타나는 횟수) / (문서의 용어 수)
#IDF = log (N / n), 여기서 N은 문서의 수이고, n은 용어 t가 나타난 문서의 수

# TF-IDF는 TF와 IDF를 곱한 값으로 점수가 높은 단어일수록 다른 문서에는 많지 않고 해당 문서에서 자주 등장하는 단어를 의미한다.


"\nTF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, \n여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 문서의 핵심어를 추출하거나, \n검색 엔진에서 검색 결과의 순위를 결정하거나, 문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.\n\nTF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, \n이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, \n이것은 그 단어가 흔하게 등장한다는 것을 의미한다. \n\n이것을 DF(문서 빈도, document frequency)라고 하며, \n이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.\nIDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 \nIDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 \n각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.\n"

In [3]:

#  정확도(Precision) 
#  재현율(Recall)

# 모델의 성능 평가 F - score
# 정확도와 재현율을 하나의 지표로 통합해서 정확성을 측정하는 방법
# 정확하면서 계속 반복시도했을때 동일한 결과가 나와야 .... 

# Precision과 Recall의 트레이드오프를 잘 통합하여 정확성을 한번에 나타내는 지표라 할 수 있다. 
# 보통 가중치를 가진 조화 평균(weighted harmonic mean)이라고도 한다. 
# * 조화 평균(Harmonic Mean)은 주어진 수들의 역수의 산술 평균을 구한 값의 역수

# F 1 score = (1 + β제곱) * (Precision * Recall) / (β제곱 * Precision + Recall)

#  β가 1인 경우 (즉 F1)를 F1 Score라고 하고, 모델의 성능 평가 지표로 많이 사용한다.
# 
#vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="word", sublinear_tf=True, ngram_range=(1,3)) #converting data to vectors
#Bad samples: 44532
#Good samples: 1265974
#Baseline Constant negative: 0.966019
#------------
#Accuracy: 0.996070
#Precision: 0.924847
#Recall: 0.960847
#F1-Score: 0.942503
#AUC: 0.987436

vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3)) #converting data to vectors
X = vectorizer.fit_transform(queries)


In [4]:
# spliting data > train, test


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #splitting data


In [5]:


# training our model 
# class_weight가 작동하는 방식 : 
# 클래스 가중치가 높다는 것은 그 클래스에 중점을 두는 것을 의미합니다. 
# 클래스 0이 클래스 1보다 더 자주 보인다면
# 클래스 0에 비해 클래스 1의 class_weight를 증가시켜야합니다. 
# 예를 들어 {0 : .1, 1 : .9}. 

# 기본값은 class_weight='balanced'
# "balanced"모드는 y 값을 사용하여 입력 데이터의 클래스 빈도에 반비례하는 가중치를 
#                        n_samples / (n_classes * np.bincount (y))
# 로 자동 조정합니다.

In [6]:


badCount = len(badQueries)
validCount = len(validQueries)

lgs = LogisticRegression(class_weight='balanced') # class_weight='balanced')
lgs.fit(X_train, y_train) #training our model


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [8]:


##############
# Evaluation #
##############

predicted = lgs.predict(X_test)


In [9]:

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))


In [10]:


auc = metrics.auc(fpr, tpr)


In [11]:


print("Bad samples: %d" % badCount)
print("Good samples: %d" % validCount)
print("Baseline Constant negative: %.6f" % (validCount / (validCount + badCount)))
print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

Bad samples: 44532
Good samples: 1265974
Baseline Constant negative: 0.966019
------------
Accuracy: 0.999252
Precision: 0.980748
Recall: 0.997268
F1-Score: 0.988939
AUC: 0.999980
